# Process gridded observation product data into variability data

__0. Is already at 5x5 deg resolution.__

__1. Use Dask to perform resampling for variability spatially.__

Resample for time periods 3-120 years (to compare with 1900-2020 observations) to generate envelopes of internal variability for each region and each model.  

- Output is a dataArray for each model with indexed by time period length (2-63 years) and lat-lon gridcell containing unforced TAS trends.

Nathan notes that these models do the best job in CONUS:  MPI-ESM, GFDL-ESM2M, CESM1-CAM5, CanESM2.

For now, I will create my code for the CESM1 and MPI models so that it can be generalized easily. I can pull some code from my climatetrend_uncertainty repository (climatetrend_uncertainty/initial_code/PIC_timeseries_preproc.ipynb).

## Code!

Most important taken from: trend_uncertainty/spatialdata_preproc.ipynb

In [21]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import glob
import dask

%matplotlib inline

### Get files.

__OBS-LENS Directory:__

In [2]:
gistemp_dir = '/glade/derecho/scratch/lenssen/data4jonah/GISTEMP_Ensemble_Aug/'
hadcrut5_dir = '/glade/derecho/scratch/lenssen/data4jonah/HadCRUT5_Ensemble_Aug/'

#### Collect GISTEMP 5x5 file paths.

In [3]:
gistemp_files = glob.glob('%s/*.nc' % gistemp_dir)
gistemp_files.sort()

#### Collect HadCRUT5 file paths.

In [4]:
hadcrut5_files = glob.glob('%s/*.nc' % hadcrut5_dir)
hadcrut5_files.sort()

### Process time series into unforced trends as a function of trend duration

Using code from:  

trend_uncertainty/PIC_trend_preproc.ipynb

__Define functions:__

In [5]:
# need a wrapper function because you can't groupby multiple dimensions
def gettrends_wrapper2(dataarray,dim,sel_dim='time.year'):
    
    dataarray[dim] = dataarray[sel_dim]

    # return dataarray.polyfit(dim=dim,deg=1)['polyfit_coefficients'].sel(degree=1)
    
    try:
        return dataarray.polyfit(dim=dim,deg=1)['polyfit_coefficients'].sel(degree=1)
    except ValueError: # Try and catch nan issues and handle them.
        print('ERROR:')
        print(dim)
        print(dataarray)
        print(dataarray.polyfit(dim=dim,deg=1))
        print(dataarray.polyfit(dim=dim,deg=1)['polyfit_coefficients'])
        print(dataarray.polyfit(dim=dim,deg=1)['polyfit_coefficients'].sel(degree=1))
        return np.nan

def calc_slopes_spatial_ann2(data,_startyr,_endyr,wrapper_func):
    '''
    Separate function for the annual trends because I was dumb.
    '''
    
    tsel = data.sel(year=slice(None,_endyr))
    
    # Test for regions with nans and mask them. polyfit breaks when there are less than 3 non-nans.
    nan_mask = ((~np.isnan(tsel)).sum(dim='year') > 2)
    tsel = tsel.where(nan_mask,np.nan)
    
    slopes_endyr_ann = wrapper_func(tsel,dim='year',sel_dim='year').drop('degree')

    # slopes_endyr = xr.concat([slopes_endyr,slopes_endyr_ann.assign_coords({'month':13}).expand_dims('month',axis=-1)],dim='month')

    slopes_endyr_ann['startyear']  = int(_startyr)
    slopes_endyr_ann['duration'] = len(tsel.year)
    
    return slopes_endyr_ann


def calc_trends_noload(data:xr.DataArray,var:str,startyear:str,endyears:np.ndarray,
                       wrapper_func,calc_slopes_func):
    
    '''
    Takes data. Computes trends. Quietly and efficiently :)
    '''
    
    slopes_by_dur = []
    
    for _endyr in endyears: # Dasking here? Yessir.

        slopes_by_dur.append(calc_slopes_func(data=data,_startyr=int(startyear),_endyr=_endyr,wrapper_func=wrapper_func))

    forced_olrslopes_monthly = xr.concat(slopes_by_dur,dim='duration')
    forced_olrslopes_monthly.name=var
    
    return forced_olrslopes_monthly

In [6]:
# Arbitrary function to apply

def gistemp_5x5_tselect(da:xr.DataArray):
    
    return da.sel(time=slice("1900-01-01","2020-12-31")).rename({'longitude':'lon','latitude':'lat'})

### For the observational uncertainty products, there are far more files to open. Need to pivot to another approach.

#### GISTEMP_5x5

In [64]:
%%time
save_path = '/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/GISTEMP_5x5/20240820/'

# Set File list
data_paths = gistemp_files

# Set var
var = 'tas'

# Set startyears and durations
startyears = [1900]
_startyear = 1900
durations = np.arange(2,121) # Only goes through 2020

gistemp_new_times = (pd.date_range("1900-01-01", freq="1M", periods=12*61)-pd.offsets.MonthBegin(1)).shift(periods=14,freq='D')

record_list = []
for _data_path in data_paths:
    
    filename = _data_path.split('/')[-1][:-3]
    
    output_string = '%s/%s.5x5degSpatialTrends.190001-202012.nc' % (save_path,filename)
    
    i = _data_path.split('_')[-1][:4]
    print(i)
    
    data = xr.open_dataset(_data_path,chunks={'time': -1})[var]

    data = gistemp_5x5_tselect(data) # Select post 1900 period

    data_ann = data.groupby('time.year').mean(dim='time')
    data_ann = data_ann.assign_coords(realization=int(i)+1).expand_dims("realization") 
    
    _endyears = _startyear + durations
    _startyear_str = str(_startyear)
    
    while len(_startyear_str) < 4: # pad with zeros
        _startyear_str = '0' + _startyear_str
    
    data_ann_slice = data_ann.sel(year=slice(_startyear,_startyear+durations[-1]))
    
    slopes_by_dur = []
    
    _endyears = _startyear + durations
    _startyear_str = str(_startyear)
    
    slopes_by_dur.append(dask.delayed(calc_trends_noload)(data=data_ann_slice,
                                                          var=var,
                                                          startyear=_startyear_str,
                                                          endyears=_endyears,
                                                          wrapper_func=gettrends_wrapper2,
                                                          calc_slopes_func=calc_slopes_spatial_ann2,
                                                          ))
    try:
        slopes_by_dur_computed = dask.compute(*slopes_by_dur)
        slopes_by_dur_computed[0].to_netcdf(output_string)
        print(output_string)
    except ValueError:
        print('ValueError for %s' % filename)
        print('var: ',var)
        print('startyear: ',_startyear_str)
        print('endyears: ',_endyears)
        break

0001


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/GISTEMP_5x5/20240820//ensembleChunk_5x5_0001.5x5degSpatialTrends.190001-202012.nc
0002


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/GISTEMP_5x5/20240820//ensembleChunk_5x5_0002.5x5degSpatialTrends.190001-202012.nc
0003


...

/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/GISTEMP_5x5/20240820//ensembleChunk_5x5_0198.5x5degSpatialTrends.190001-202012.nc
0199


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/GISTEMP_5x5/20240820//ensembleChunk_5x5_0199.5x5degSpatialTrends.190001-202012.nc
0200


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/GISTEMP_5x5/20240820//ensembleChunk_5x5_0200.5x5degSpatialTrends.190001-202012.nc
CPU times: user 8min 58s, sys: 5.84 s, total: 9min 4s
Wall time: 9min 35s


#### HadCrut5

In [24]:
hadcrut_test = xr.open_dataset(hadcrut5_files[0])

In [66]:
%%time
save_path = '/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/HadCRUT5/20240820/'

# Set File list
data_paths = hadcrut5_files

# Set var
var = 'tas'

# Set startyears and durations
startyears = [1900]
_startyear = 1900
durations = np.arange(2,121) # Only goes through 2020

record_list = []
for _data_path in data_paths:
        
    data = xr.open_dataset(_data_path,chunks={'time': -1})[var]

    data = data.sel(time=slice('1900-01-01',None)) # Select post 1900 period

    data_ann = data.groupby('time.year').mean(dim='time')
    
    _endyears = _startyear + durations
    _startyear_str = str(_startyear)
    
    while len(_startyear_str) < 4: # pad with zeros
        _startyear_str = '0' + _startyear_str
    
    data_ann_slice = data_ann.sel(year=slice(_startyear,_startyear+durations[-1]))
    
    slopes_by_dur = []
    
    _endyears = _startyear + durations
    _startyear_str = str(_startyear)
    
    slopes_by_dur.append(dask.delayed(calc_trends_noload)(data=data_ann_slice,
                                                          var=var,
                                                          startyear=_startyear_str,
                                                          endyears=_endyears,
                                                          wrapper_func=gettrends_wrapper2,
                                                          calc_slopes_func=calc_slopes_spatial_ann2,
                                                          ))
    
    slopes_by_dur_computed = dask.compute(*slopes_by_dur)
    
    filename = _data_path.split('/')[-1][:-3]
    
    output_string = '%s/%s.5x5degSpatialTrends.190001-202012.nc' % (save_path,filename)
    print(output_string)
    
    slopes_by_dur_computed_final = slopes_by_dur_computed[0].assign_coords({'realization':data.realization}).expand_dims('realization',axis=-1)    
    
    slopes_by_dur_computed_final.to_netcdf(output_string)


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/HadCRUT5/20240820//HadCRUT.5.0.2.0.analysis.anomalies.1.5x5degSpatialTrends.190001-202012.nc


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/HadCRUT5/20240820//HadCRUT.5.0.2.0.analysis.anomalies.10.5x5degSpatialTrends.190001-202012.nc


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


...

/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/HadCRUT5/20240820//HadCRUT.5.0.2.0.analysis.anomalies.97.5x5degSpatialTrends.190001-202012.nc


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/HadCRUT5/20240820//HadCRUT.5.0.2.0.analysis.anomalies.98.5x5degSpatialTrends.190001-202012.nc


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


/glade/work/jonahshaw/trend_uncertainty/nathan/OBS_LENS/HadCRUT5/20240820//HadCRUT.5.0.2.0.analysis.anomalies.99.5x5degSpatialTrends.190001-202012.nc
CPU times: user 14min 36s, sys: 5min 20s, total: 19min 57s
Wall time: 21min 50s
